In [ ]:
# 이전 IBCF 코드!!! (시간 너무 걸림)

def recom_category(user_id, n_items=20, neighbor_size=28):
    viewed_categories = view_matrix.loc[user_id].copy().sort_values(ascending=False)[:100]
    for category in view_matrix:
        # 모든 카테고리에 대해서 예측값을 만들어줍니다.
        user_category.loc[category] = IBCF_model(user_id, category, neighbor_size)
        
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    # 예측값을 sorting하고.... n_items만큼만 뽑아줍니다.
    return category_sort

def IBCF_model(user, category, neighbor_size):
    """
        예측값을 만들어주는 Model입니당
        특정 유저 id와 특정 카테고리 no를 토대로 예측값을 만들어줍니다.
        item_similarity: 미리 만들어둔 코사인 유사도로 계산한 category의 유사도입니다.
        user_category: 해당 user의 카테고리별 조회수입니다.
    """
    sim_scores = item_similarity[category]
    # 해당 카테고리와 다른 카테고리의 유사도들입니다.
    user_viewing = user_category.T
    # 해당 user의 카테고리별 조회수입니다. (계산을 위해 전치행렬로 만듭니다.)
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    # user가 조회하지 않은 카테고리의 index를 찾습니다.
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)
    # user_viewing과 sim_scores에서 비어있는거 제거합니다. (계산속도 향상용)
    
    neighbor_size = min(neighbor_size, len(sim_scores))
    sim_scores = np.array(sim_scores)
    user_viewing = np.array(user_viewing)
    user_idx = np.argsort(sim_scores)
    sim_scores = sim_scores[user_idx][-neighbor_size:]
    user_viewing = user_viewing[user_idx][-neighbor_size:]
    pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    # 예측값은 유저의 조회 기록을 해당 카테고리와의 유사도로 가중평균한 값입니다.
    
    return pred_viewing

test_user = {
    'user_id': 90001,
    'gender': 'M',
    'age': 17
}

user_category = view_matrix.loc[test_user['user_id']].copy()
# test_user의 카테고리별 조회수입니다.
pred_user_category = recom_category(user_id=test_user['user_id'], n_items=10)
# 추천 카테고리를 가져옵니다. (위 함수 참조)
recommend_category = pd.DataFrame(pred_user_category).reset_index()
recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
# 카테고리 no를 카테고리명으로 바꿉니다.
print(recommend_category)

In [ ]:
tester = 90001
user_category = view_matrix.loc[tester].copy()
print(user_category.sort_values(ascending=False))
# test_user의 카테고리별 조회수입니다.
pred_user_category = recom_category(user_id=tester, n_items=10)
# 추천 카테고리를 가져옵니다. (위 함수 참조)
recommend_category = pd.DataFrame(pred_user_category).reset_index()
recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
# 카테고리 no를 카테고리명으로 바꿉니다.
print(new_df_users[new_df_users['user'] == tester])
print(recommend_category)

In [ ]:
# 버그 수정...? 너무 이상함
def recom_category(user_id, n_items=20, neighbor_size=28):
    for category in view_matrix:
        # 모든 카테고리에 대해서 예측값을 만들어줍니다.
        user_category.loc[category] = IBCF_model(user_id, category, neighbor_size)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    # 예측값을 sorting하고.... n_items만큼만 뽑아줍니다.
    return category_sort

def IBCF_model(user, category, neighbor_size):
    """
        예측값을 만들어주는 Model입니당
        특정 유저 id와 특정 카테고리 no를 토대로 예측값을 만들어줍니다.
        item_similarity: 미리 만들어둔 코사인 유사도로 계산한 category의 유사도입니다.
        user_category: 해당 user의 카테고리별 조회수입니다.
    """
    sim_scores = item_similarity[category]
    sim_scores[category] = 0
    # 해당 카테고리와 다른 카테고리의 유사도들입니다.
    user_viewing = source_category.T
    # 해당 user의 카테고리별 조회수입니다. (계산을 위해 전치행렬로 만듭니다.)
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    # user가 조회하지 않은 카테고리의 index를 찾습니다.
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)
    # user_viewing과 sim_scores에서 비어있는거 제거합니다. (계산속도 향상용)

    if sim_scores.sum() >= 1.5:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        user_viewing = np.array(user_viewing)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        user_viewing = user_viewing[user_idx][-neighbor_size:]
        pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    else:
        pred_viewing = 0
                
    return pred_viewing

test_user = {
    'user_id': 1,
    'gender': 'F',
    'age': 26
}

# user_category = view_matrix.loc[test_user['user_id']].copy()
# # test_user의 카테고리별 조회수입니다.
# pred_user_category = recom_category(user_id=test_user['user_id'], n_items=10, neighbor_size=28)
# # 추천 카테고리를 가져옵니다. (위 함수 참조)

test = 90001
user_category = view_matrix.loc[test].copy()
source_category = view_matrix.loc[test].copy()

pred_user_category = recom_category(user_id=test, n_items=10, neighbor_size=28)

recommend_category = pd.DataFrame(pred_user_category).reset_index()
recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
# 카테고리 no를 카테고리명으로 바꿉니다.
print(recommend_category)

In [ ]:
# def recom_category(user_id, n_items=20, neighbor_size=28):
#     for category in view_matrix:
#         predictions.loc[category] = IBCF_model(user_id, category, neighbor_size)
#     print(predictions)
#     category_sort = predictions.sort_values(by='count', ascending=False)[:n_items]
#     return category_sort

# def IBCF_model(user, category, neighbor_size):
#     sim_scores = item_similarity[category]
#     # 해당 카테고리와 다른 카테고리의 유사도들입니다.
#     user_viewing = user_category.T
#     # 해당 user의 카테고리별 조회수입니다. (계산을 위해 전치행렬로 만듭니다.)
#     non_viewing_idx = user_viewing[user_viewing.isnull()].index
#     # user가 조회하지 않은 카테고리의 index를 찾습니다.
#     user_viewing = user_viewing.dropna()
#     sim_scores = sim_scores.drop(non_viewing_idx)
#     # user_viewing과 sim_scores에서 비어있는거 제거합니다. (계산속도 향상용)

#     neighbor_size = min(neighbor_size, len(sim_scores))
#     sim_scores = np.array(sim_scores)
#     user_viewing = np.array(user_viewing)
#     user_idx = np.argsort(sim_scores)
#     sim_scores = sim_scores[user_idx][-neighbor_size:]
#     user_viewing = user_viewing[user_idx][-neighbor_size:]
# #     print(category, sim_scores.sum())
# #     print(sim_scores)
# #     print(user_viewing)
#     print(category, sim_scores.sum())
#     if sim_scores.sum():
#         pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
#     else:
#         pred_viewing = 1
# #     print(pred_viewing)
# #     print('-' * 80)

#     return pred_viewing

# user_category = view_matrix.loc[1].copy()
# predictions = pd.DataFrame(0, index=view_matrix.columns, columns=['count'])
# # test_user의 카테고리별 조회수입니다.
# pred_user_category = recom_category(user_id=1, n_items=10, neighbor_size=423)
# print(pred_user_category)
# # # 추천 카테고리를 가져옵니다. (위 함수 참조)
# recommend_category = pd.DataFrame(pred_user_category).reset_index()
# recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
# # # 카테고리 no를 카테고리명으로 바꿉니다.
# print(recommend_category)

In [ ]:
# 이건 피어슨 상관계수로 한번 해봤습니다..........
item_similarity2 = pd.read_pickle('recom_data/item_similarity_pearson.pkl')

def recom_category(user_id, n_items=20):
    for category in view_matrix:
        user_category.loc[category] = IBCF_model(user_id, category)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    return category_sort

def IBCF_model(user, category):
    sim_scores = item_similarity2[category]
    user_viewing = user_category.T
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)
    pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    return pred_viewing

user_category = view_matrix.loc[test_user['user_id']].copy()

pred_user_category = recom_category(user_id=test_user['user_id'], n_items=10)
recommend_category = pd.DataFrame(pred_user_category).reset_index()
recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
print(recommend_category)